In [1]:
from typing import *

In [67]:
import pandas as pd


def find_consistently_improving_employees(employees: pd.DataFrame, performance_reviews: pd.DataFrame) -> pd.DataFrame:

    # We rank rating by date for each employee
    performance_reviews['rank'] = (performance_reviews.
                                   groupby('employee_id')['review_date'].rank(ascending=False))

    # We filter for most recent three
    df = performance_reviews[performance_reviews['rank'] <= 3]

    # We create a pivot table with columns for each rating
    df = df.pivot(index="employee_id", columns="rank",
                  values="rating").reset_index()

    # We filter for increasing ratings, and we compute score
    df = df.dropna()[(df[1] > df[2]) & (df[2] > df[3])]
    df['improvement_score'] = df[1] - df[3]

    # We merge employee names, sort rows, and rearrange columns
    return (df.merge(employees)
            .sort_values(['improvement_score', 'name'], ascending=[0, 1])
            .iloc[:, [0, 5, 4]])

In [68]:
employees = pd.DataFrame({
    "employee_id": [1,2,3,4,5],
    "name": ['Alice Johnson','Bob Smith', 'Carol Davis', 'David Wilson', 'Emma Brown']
})

performance_reviews = pd.DataFrame({
    "review_id": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
    "employee_id": [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,5,5],
    "review_date": ['2023-01-15','2023-04-15','2023-07-15','2023-10-15','2023-02-01','2023-05-01','2023-08-01','2023-11-01','2023-03-10','2023-06-10','2023-09-10','2023-12-10','2023-01-20','2023-04-20','2023-07-20','2023-02-15','2023-05-15'],
    "rating": [2,3,4,5,3,2,4,5,1,2,3,4,4,4,4,3,2]
})
performance_reviews["review_date"] = pd.to_datetime(performance_reviews['review_date'])

# print(employees)
# print()
# print(performance_reviews)
# print()

print(find_consistently_improving_employees(employees, performance_reviews))

   employee_id           name  improvement_score
1            2      Bob Smith                3.0
0            1  Alice Johnson                2.0
2            3    Carol Davis                2.0


C:\Users\manan\AppData\Local\Temp\ipykernel_21976\3015031968.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.dropna()[(df[1] > df[2]) & (df[2] > df[3])]
